# Class level relationships

## Generalization

We need a target project rather than a single file.

In [ ]:
# Clone a target project. This time, we try Mojang/brigadier, a MineCraft program.
!git clone git@github.com:Mojang/brigadier.git

In [1]:
# First, we need to read many java files.
# from os.path import isfile, join
import os

path = "./brigadier/src/main/java/**/*.java"

from glob import glob


for filename in glob(path, recursive=True):
    print(filename)

./brigadier/src/main/java/com/mojang/brigadier/SingleRedirectModifier.java
./brigadier/src/main/java/com/mojang/brigadier/Message.java
./brigadier/src/main/java/com/mojang/brigadier/ResultConsumer.java
./brigadier/src/main/java/com/mojang/brigadier/Command.java
./brigadier/src/main/java/com/mojang/brigadier/StringReader.java
./brigadier/src/main/java/com/mojang/brigadier/ParseResults.java
./brigadier/src/main/java/com/mojang/brigadier/ImmutableStringReader.java
./brigadier/src/main/java/com/mojang/brigadier/RedirectModifier.java
./brigadier/src/main/java/com/mojang/brigadier/AmbiguityConsumer.java
./brigadier/src/main/java/com/mojang/brigadier/LiteralMessage.java
./brigadier/src/main/java/com/mojang/brigadier/CommandDispatcher.java
./brigadier/src/main/java/com/mojang/brigadier/tree/ArgumentCommandNode.java
./brigadier/src/main/java/com/mojang/brigadier/tree/LiteralCommandNode.java
./brigadier/src/main/java/com/mojang/brigadier/tree/CommandNode.java
./brigadier/src/main/java/com/mojang

In [2]:
javafiles = list(glob(path, recursive=True))
len(javafiles)

47

In [3]:
## Parser function
import javalang

def getAST(path: str):
    with open(path, "r") as f:
        code = f.read()
    return javalang.parse.parse(code)

## Get AST one-by-one

In [24]:
# print(javafiles[0])
myast = getAST(javafiles[14])
# myast

## Detect `extends` and `implements`

In [4]:
gen_real_dict = {}

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
            gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                         "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                         "type": "class"}
#             print("=====")
#             print("Class:", node.name)
#             print("Extends:", node.extends)
#             print("Implements:", node.implements)
        elif isinstance(node, javalang.tree.InterfaceDeclaration):
            gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                         "type": "interface"}
#             print("=====")
#             print("Interface:", node.name)
#             print("Extends:", node.extends)
        elif isinstance(node, javalang.tree.AnnotationDeclaration):
            gen_real_dict[node.name] = {}
#             print("=====")
#             print("Annotation:", node.name)
        elif isinstance(node, javalang.tree.EnumDeclaration):
            gen_real_dict[node.name] = { "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                         "type": "enum"}
#             print("=====")
#             print("Enum:", node.name)
#             print("Implements:", node.implements)    

In [45]:
# gen_real_dict
for key, value in gen_real_dict.items():
    if value["type"] == "class":
        if value["extends"] is not None:
            gen_real_dict[key]["dangling"] = False
            if value["extends"] in gen_real_dict:
                gen_real_dict[value["extends"]]["dangling"] = False
        if value["implements"] is not None:
            for aiterface in value["implements"]:
                gen_real_dict[key]["dangling"] = False
                if aiterface in gen_real_dict:
                    gen_real_dict[aiterface]["dangling"] = False
    elif value["type"] == "interface":
        if value["extends"] is not None:
            gen_real_dict[key]["dangling"] = False
            if value["extends"] in gen_real_dict:
                gen_real_dict[value["extends"]]["dangling"] = False
#     if value["implements"] is not None and len(value["implements"]) > 1:
#         print(f"{key}:", value["implements"])

for key, value in gen_real_dict.items():
    if "dangling" not in value:
        print(f"{key} is dangling")

SingleRedirectModifier is dangling
ResultConsumer is dangling
Command is dangling
ParseResults is dangling
RedirectModifier is dangling
AmbiguityConsumer is dangling
CommandDispatcher is dangling
ParsedArgument is dangling
SuggestionContext is dangling
CommandContextBuilder is dangling
ContextChain is dangling
Stage is dangling
StringRange is dangling
ParsedCommandNode is dangling
CommandContext is dangling
SuggestionProvider is dangling
Suggestions is dangling
SuggestionsBuilder is dangling
Function is dangling
StringType is dangling


### Excercise: Identify dangling interfaces
It should be useful if we can figure out interfaces, which are not implemented by any other classes or is not a parent interface of anohter interface.

In [5]:
gen_real_dict

{'SingleRedirectModifier': {'extends': None, 'type': 'interface'},
 'Message': {'extends': None, 'type': 'interface'},
 'ResultConsumer': {'extends': None, 'type': 'interface'},
 'Command': {'extends': None, 'type': 'interface'},
 'StringReader': {'extends': None,
  'implements': ['ImmutableStringReader'],
  'type': 'class'},
 'ParseResults': {'extends': None, 'implements': None, 'type': 'class'},
 'ImmutableStringReader': {'extends': None, 'type': 'interface'},
 'RedirectModifier': {'extends': None, 'type': 'interface'},
 'AmbiguityConsumer': {'extends': None, 'type': 'interface'},
 'LiteralMessage': {'extends': None,
  'implements': ['Message'],
  'type': 'class'},
 'CommandDispatcher': {'extends': None, 'implements': None, 'type': 'class'},
 'ArgumentCommandNode': {'extends': 'CommandNode',
  'implements': None,
  'type': 'class'},
 'LiteralCommandNode': {'extends': 'CommandNode',
  'implements': None,
  'type': 'class'},
 'CommandNode': {'extends': None,
  'implements': ['Comparabl

## Store generalization relationships in a data structure

In [36]:
for key, value in gen_real_dict.items():
    if value["extends"] is not None:
        parent = value["extends"]
        print(f"{key} -> {parent}")

ArgumentCommandNode -> CommandNode
LiteralCommandNode -> CommandNode
RootCommandNode -> CommandNode
IntegerSuggestion -> Suggestion
CommandSyntaxException -> Exception
RequiredArgumentBuilder -> ArgumentBuilder
LiteralArgumentBuilder -> ArgumentBuilder


### Excercise: making inverse relationships of generalization
Currently, we have `subclass -> parentclass`. We can computer `parentclass -> subclass`.

In addition, we can create a forest of class hierarchy in a project by using the above information.

# Association

## Composition
It is quite challenging to figure out compositions in Java as it needs dynamic information.

## Aggregation

#### Fields (member variables)

In [8]:
classes = {}

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
           classes[node.name] = {"ast_node": node}

In [18]:
for cname, props in classes.items():
    fields = {}
    for field in props["ast_node"].fields:
        type_names = []
        if isinstance(field.type, javalang.tree.BasicType):
            type_names.append(field.type.name)
        elif isinstance(field.type, javalang.tree.ReferenceType):
            type_names.append(field.type.name)
            if field.type.arguments is not None:
                for path, node in javalang.ast.walk_tree(field.type.arguments):
                    if isinstance(node, javalang.tree.ReferenceType):
                        type_names.append(node.name)
#         print("type:", field.type)
#         print("modifier:", field.modifiers)
#         print("annotations:", field.annotations)
        for decl in field.declarators:
            fields[decl.name] = {
                "static": True if "static" in field.modifiers else False,
                "types": type_names
            }
        print(type_names)
            
#             print("declarated name:", decl.name)

['char']
['char']
['char']
['String']
['int']
['CommandContextBuilder', 'S']
['Map', 'CommandNode', 'S', 'CommandSyntaxException']
['ImmutableStringReader']
['String']
['String']
['char']
['String']
['String']
['String']
['String']
['String']
['RootCommandNode', 'S']
['Predicate', 'CommandNode', 'S']
['ResultConsumer', 'S']
['String']
['String']
['String']
['ArgumentType', 'T']
['SuggestionProvider', 'S']
['String']
['String']
['Map', 'String', 'CommandNode', 'S']
['Map', 'String', 'LiteralCommandNode', 'S']
['Map', 'String', 'ArgumentCommandNode', 'S']
['Predicate', 'S']
['CommandNode', 'S']
['RedirectModifier', 'S']
['boolean']
['Command', 'S']
['StringRange']
['T']
['CommandNode', 'S']
['int']
['Map', 'String', 'ParsedArgument', 'S']
['CommandNode', 'S']
['List', 'ParsedCommandNode', 'S']
['CommandDispatcher', 'S']
['S']
['Command', 'S']
['CommandContextBuilder', 'S']
['StringRange']
['RedirectModifier', 'S']
['boolean']
['List', 'CommandContext', 'S']
['CommandContext', 'S']
['Cont